# ようこそ

maDMPへようこそ。ここでは、DMPに入力いただいた研究分野等情報に基づき、研究活動の支援、およびデータ管理品質向上のためにデータガバナンス機能が提供するワークフローを最適化します。

![maDMP_to_workflow](https://raw.githubusercontent.com/ivis-kuwata/maDMP-template/main/images/maDMP_to_workflow.jpg)

このmaDMPには、データガバナンス機能で作成いただいたDMPの情報の中で、ワークフローの最適化に必要な情報が既に埋め込まれています。以下を順番に実行することで、システムがDMP情報を活用した最適化処理を行います。

In [ ]:
# DMP情報
field = '%v'
dataSize = '%v'
datasetStructure = '%v'

In [ ]:
# ワークフローテンプレート取得
clone_url = 'https://github.com/ivis-kuwata/workflow-template'+'.git'
workflow_path = '/home/jovyan/WORKFLOW'
!git clone $clone_url $workflow_path
!rm -rf $workflow_path/.git

In [ ]:
# dmp.jsonに"fields"プロパティがある想定
import os
import glob
import shutil

path_flows = os.path.join('WORKFLOW', 'FLOW')

templates = glob.glob(os.path.join(path_flows, '**'), recursive=True)

# 選択外の分野のセクション群を削除
for tmpl in templates:
    file = os.path.basename(tmpl)
    if not os.path.isdir(tmpl) and os.path.splitext(file)[1] == '.ipynb':
        if 'base_' not in file and field not in file:
            os.remove(tmpl)
            
print('最適化処理が完了しました。次の手順へお進みください。')

In [ ]:
# モニタリング準備
import os
import json

path_params = os.path.join('WORKFLOW', 'FLOW', 'param_files', 'monitoring_params.json')
monitoring_dict = {
    'dataSize': dataSize

    # モニタリング項目を追加する場合は以下に追記してください
    # e.g. 'dirStructure': 'withCode'
}

with open(path_params, 'w') as f:
    json.dump(monitoring_dict, f, indent=4)


# ワークフロー全体の関連図と目次を生成する

次に、最適化したワークフロー全体が俯瞰できる図を生成します。  
以下を実行することで、システムがワークフロー関連図、および目次を作成します。

◆◆◆開発メモ◆◆◆

*.diagに分野固有のセクションはまだ反映されていない？

In [ ]:
import os
import nbformat

flow_path = 'WORKFLOW/FLOW/base_FLOW.ipynb'
rcos_binder_url = 'https://jupyter.cs.rcos.nii.ac.jp/'
flow_path_notebook = 'notebooks/WORKFLOW/FLOW/base_FLOW.ipynb'
# flow_path_lab = 'lab/tree/WORKFLOW/FLOW/base_FLOW.ipynb'

nb = nbformat.read(flow_path, 4)

title1 = """## ワークフロー図を作成する

以下のセルを実行してワークフロー図を作成できます。"""

code = """import os
from IPython.display import SVG

nb_utils_path   = os.path.join(os.environ['HOME'], 'WORKFLOW/FLOW/util/scripts/nb_utils.py')

basic_path      = os.path.join(os.environ['HOME'], 'WORKFLOW/images')
basic_path_diag = os.path.join(basic_path, 'notebooks.diag')
basic_path_svg  = os.path.join(basic_path, 'notebooks.svg')

os.chdir(os.environ['HOME'])
!python3 $nb_utils_path $basic_path_diag

SVG(filename=basic_path_svg)"""

title2 = """ワークフロー図の左のノードはワークフローのフェーズとその流れを表しており、フェーズの右側にあるノードは個々のワークフローを表しています。  
各ワークフローを実行するには、ワークフローの各ノードのリンクをクリックしてください。"""

nb["cells"] += [
    nbformat.v4.new_markdown_cell(title1),
    nbformat.v4.new_code_cell(code),
    nbformat.v4.new_markdown_cell(title2)
]

with open(flow_path, "w") as f:
    nbformat.write(nb, f)

print('ワークフローの生成が完了しました。以下のURLをクリックすることで')
print('ワークフロー全体図を閲覧できるページに遷移します。')
print(rcos_binder_url + os.environ['JUPYTERHUB_SERVICE_PREFIX'] + flow_path_notebook)